In [37]:
# Import libraries

import pandas as pd
import csv, sys
import nltk
from nltk.corpus import stopwords
import string
from textblob import TextBlob
from nltk.stem import PorterStemmer
from collections import defaultdict

In [2]:
# Read the initial csv file and remove unnamed column
df = pd.read_csv('C:\\Users\\Olya\Documents\\Various documents\\Masters\\La Sapienza\\ADM\\Homework\\Homework 3\\airbnb-property-data-from-texas\\Airbnb_Texas_Rentals.csv')
df = df.drop('Unnamed: 0', axis = 1)

In [3]:
# Create one tsv file per each row
df = pd.read_csv('C:\\Users\\Olya\Documents\\Various documents\\Masters\\La Sapienza\\ADM\\Homework\\Homework 3\\airbnb-property-data-from-texas\\Airbnb_Texas_Rentals.csv')
df = df.drop('Unnamed: 0', axis = 1)
for i in range(df.index.max()+1):
        new = open('doc_'+str(i)+'.tsv', 'w')
        for j in range(9):
               new.write('%s\t' %df.iloc[i, j])
        new.close()
        if i > 2: # for the moment just making 4 files
            break

In [6]:
# Create function to remove stop words from specified column from data frame
# in our case we'll call it for the title (column 4) and description (column 7)
def remove_stopwords(df, n):
    #remove stopwords
    stop = stopwords.words('english')
    df[n] = df[n].apply(lambda x: " ".join(x for x in x.split() if x not in stop)) 
    return df

In [7]:
# Create function to remove punctuation from specified column from data frame
# in our case we'll call it for the title (column 4) and description (column 7)
def remove_punctuation(df, n):
    df[n] = df[n].str.replace('[^\w\s]','') 
    return df

In [8]:
# Create function to put everything to lowercase from specified column from data frame
# in our case we'll call it for the title (column 4) and description (column 7)
def to_lowercase(df, n):
    df[n] = df[n].apply(lambda x: " ".join(x.lower() for x in x.split())) 
    return df

In [22]:
# Create function to do stemming from specified column from data frame
# in our case we'll call it for the title (column 4) and description (column 7)
def stemming(df, n):
    st = PorterStemmer()
    df[n] = df[n].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
    return df

In [23]:
# Create a function which makes unique words from max 2 given columns from data frame
# second argument can be omitted
# !!!!! what shell we do with numbers? keep them as a word or remove them?

def unique_words(df, n, m):
    a=set(" ".join(df[n]).split(" "))
    if m == None:
          b=set() # empty set for the second argument
    else:
        b=set(" ".join(df[m]).split(" "))
    # return unique words
    return a.union(b)

In [59]:
def inverted_idx(inverted_idx, w, i):
    for w in doc_unique_words:
        inverted_idx[w].append(i)
    return inverted_idx

In [86]:
# dictionary of all unique words appearing in documents
vocab={}
# word index of all unique words appearing in documents
word_idx=defaultdict(list)

# inverted index
inverted_idx={}

for i in range(0,3): #len(df.index)+1
    #temoporaly create a dataframe from our tsv files
    tsv = pd.read_csv('doc_{}.tsv'.format(i), sep='\t', encoding = 'ISO-8859-1', header = None)
    
    # replacing \n with the space
    tsv = tsv.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\\n',  ' ', regex=True)
    
    # replacing / with the space
    tsv = tsv.replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'/',  ' ', regex=True)
    
    # Data pre-processing
    tsv=remove_stopwords(tsv, 4) # remove stop words in title
    tsv=remove_stopwords(tsv, 7) # remove stop words in description
    tsv=remove_punctuation(tsv, 4) # remove punctuation in title
    tsv=remove_punctuation(tsv, 7) # remove punctuation in description
    tsv=to_lowercase(tsv, 4) # put title into lower case
    tsv=to_lowercase(tsv, 7) # put title into lower case
    tsv=stemming(tsv, 4) # make stemming in title
    tsv=stemming(tsv, 7) # make stemming in description
    
    # make a set of unique words from description and title of a document
    doc_unique_words=unique_words(tsv,4,7)
    
    # update current vocabulary with the new words found in a document
    for w in doc_unique_words:
        if w in vocab.values(): # if word is already in general vocabulary skip it
            continue
        else:
            if any(vocab)==False: #if vocabulary is empty then define initial integer key for the first word
                v_idx=10000
                vocab.update( { v_idx: w} ) # add a word to the vocabulary
            else: # if vocabulary already has words, take the first available key (max(key)+1) for the next word
                v_idx=max(list(vocab.keys()))+1
                vocab.update( { v_idx: w} ) # add a word to the vocabulary
                
    # create index dictionary which for each word will have list of document numbers which contain that word
    for w in doc_unique_words:
        word_idx[w].append(i)
word_idx=dict(word_idx)

# Inverted index for all the words - dictionary with word_id as a key and list of the id of the documents that contain the word
inverted_idx=dict((key,word_idx[value]) for (key, value) in vocab.items())

with open('inverted_index.csv', 'w') as f:
    w = csv.DictWriter(f, inverted_idx.keys())
    w.writeheader()
    w.writerow(inverted_idx)

with open('vocabulary.csv', 'w') as f:
    w = csv.DictWriter(f, vocab.keys())
    w.writeheader()
    w.writerow(vocab)

In [87]:
vocab

{10000: '2',
 10001: 'drop',
 10002: 'bedroom',
 10003: 'bed',
 10004: 'trip',
 10005: 'airport',
 10006: 'iah',
 10007: '10min',
 10008: '10',
 10009: 'avail',
 10010: 'addit',
 10011: 'queen',
 10012: 'room',
 10013: 'bathroom',
 10014: 'pickup',
 10015: 'privat',
 10016: 'sofa',
 10017: 'guest',
 10018: 'welcom',
 10019: 'detach',
 10020: 'floor',
 10021: 'stay',
 10022: 'second',
 10023: 'anoth',
 10024: 'open',
 10025: 'remodel',
 10026: '281',
 10027: 'amaz',
 10028: 'prestigi',
 10029: 'plan',
 10030: 'nw',
 10031: 'european',
 10032: '3',
 10033: 'decor',
 10034: 'home',
 10035: 'conveni',
 10036: 'driveway',
 10037: 'downtown',
 10038: 'gardentv',
 10039: 'full',
 10040: 'car',
 10041: 'street',
 10042: 'season',
 10043: '4',
 10044: 'park',
 10045: 'hardwood',
 10046: 'fulli',
 10047: 'inspir',
 10048: 'design',
 10049: 'sleep',
 10050: 'locat',
 10051: 'line',
 10052: 'kitchen',
 10053: 'quiet',
 10054: 'hous',
 10055: 'upscal',
 10056: 'tree',
 10057: '410',
 10058: 'neighb